## I. Load Data using Web API

Many websites (Twitter, Facebook, Kaggle, Reddit, ...) offer Application Programming Interfaces (APIs) which provide access to data on their web server. Today we will use Twitter API to download and analyze some tweets.

**Get started with Twitter API:**

1. Sign up on Twitter.
2. Apply for [developer access](https://developer.twitter.com/en/apps).
3. Create a Twitter app. (You can use lehman website for the placeholder on the urls)

In [1]:
# Install tweepy package for Python
!pip install tweepy

In [2]:
import tweepy

To learn more about the tweepy API, visit the [offical documentation](http://docs.tweepy.org/en/latest/getting_started.html)

In [ ]:
# Copy and paste tokens from "Keys and Access Tokens" tab


In [7]:
consumer_key = "1lbjB5z4RR4lh92pKQP4R1rwk"
consumer_secret = "WsLeHFw7Z6azT9bdKOGrMkkDHXFhOpXVJXHyjZHXRUIvOtRB6B"
access_token = "1189580463914061825-ZgJUr2miaSLfCgNPIvmHJOmoT2roqD"
access_token_secret = "0Hgqf5R8M5OTJEPwCZxduslOxeSmPE42zXXLCqmpYs06M"

In [8]:
# User authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [9]:
# Create API object to access twitter data
api = tweepy.API(auth)

## Task 1: Retrieve tweets from timeline

In [10]:
# My timeline
public_tweets = api.home_timeline(tweet_mode = "extended")

In [13]:
# Look into one tweet data
tweet = public_tweets[5]
tweet

Status(_api=<tweepy.api.API object at 0x000002A16CFB5470>, _json={'created_at': 'Wed Oct 30 18:44:32 +0000 2019', 'id': 1189614078534311936, 'id_str': '1189614078534311936', 'full_text': '....Your current Governor, John Bel Edwards, has done a poor job with economic development (last in nation) and auto insurance (highest in nation) wants to raise your taxes BIG - vote for Eddie starting Saturday!', 'truncated': False, 'display_text_range': [0, 212], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': 1189614076743376897, 'in_reply_to_status_id_str': '1189614076743376897', 'in_reply_to_user_id': 25073877, 'in_reply_to_user_id_str': '25073877', 'in_reply_to_screen_name': 'realDonaldTrump', 'user': {'id': 25073877, 'id_str': '25073877', 'name': 'Donald J. Trump', 'screen_name': 'realDonaldTrump', 'location': 'Washington, DC', 'description': '4

In [14]:
# the _json attribute contains info of the tweet
tweet._json

{'created_at': 'Wed Oct 30 18:44:32 +0000 2019',
 'id': 1189614078534311936,
 'id_str': '1189614078534311936',
 'full_text': '....Your current Governor, John Bel Edwards, has done a poor job with economic development (last in nation) and auto insurance (highest in nation) wants to raise your taxes BIG - vote for Eddie starting Saturday!',
 'truncated': False,
 'display_text_range': [0, 212],
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': 1189614076743376897,
 'in_reply_to_status_id_str': '1189614076743376897',
 'in_reply_to_user_id': 25073877,
 'in_reply_to_user_id_str': '25073877',
 'in_reply_to_screen_name': 'realDonaldTrump',
 'user': {'id': 25073877,
  'id_str': '25073877',
  'name': 'Donald J. Trump',
  'screen_name': 'realDonaldTrump',
  'location': 'Washington, DC',
  'description': '45th President of the United States of Amer

In [17]:
# Use the _json attribute to infer some information
obj = tweet._json
# obj['created_at']
obj['user']['name']

'Donald J. Trump'

In [18]:
# Find specific info
print(tweet.full_text)
print(tweet.author.name)
print(tweet.created_at)
print(tweet.author.location)

....Your current Governor, John Bel Edwards, has done a poor job with economic development (last in nation) and auto insurance (highest in nation) wants to raise your taxes BIG - vote for Eddie starting Saturday!
Donald J. Trump
2019-10-30 18:44:32
Washington, DC


In [19]:
# Use api.home_timeline without setting tweet_mode="extended" may result in truncated messages.
tweets2 = api.home_timeline()

In [20]:
for idx in range(10):
    print('-' * 80)
    tweet = tweets2[idx]
    print(tweet.author.name)
    print(tweet.text)

--------------------------------------------------------------------------------
Donald J. Trump
We will be having a big rally for Governor @MattBevin on Monday night in Kentucky. Get your tickets today! https://t.co/uoPhCLGHpH
--------------------------------------------------------------------------------
Real Madrid C.F.⚽
🏃‍♂️👐🔥 @thibautcourtois 
#RMLiga | #HalaMadrid https://t.co/9EK9kdjpRt
--------------------------------------------------------------------------------
Real Madrid C.F.⚽
📺 DIRECTO BERNABÉU 🏟
¡Calentamiento previo al encuentro 🆚 @CDLeganes!
#RMTV | #HalaMadrid   https://t.co/WJunuc7wlX
--------------------------------------------------------------------------------
LehmanCMP464
First tweet!
--------------------------------------------------------------------------------
Real Madrid C.F.⚽
🏟👋 ¡La llegada del equipo al Bernabéu! Ya falta menos para el saque inicial 🆚 @CDLeganes... 
#RMLiga | #HalaMadrid https://t.co/rjfbc249VI
------------------------------------------

## Task 2: Retrieve Tweets from Another User

In [21]:
name = "nytimes"
tweetCount = 20
results = api.user_timeline(id=name, count=tweetCount, tweet_mode = "extended")

In [22]:
for tweet in results:
    print('-' * 80)
    print(tweet.full_text)

--------------------------------------------------------------------------------
RT @TheWeekly: To win over skeptical conservative voters, Donald Trump had to assure them he would appoint strict constructionist judges. #…
--------------------------------------------------------------------------------
Georgetown University announced on Tuesday that it would raise about $400,000 a year to benefit the descendants of the 272 enslaved people who were sold to help keep the college afloat nearly 2 centuries ago, officials said https://t.co/HRQ6D9I7Pb
--------------------------------------------------------------------------------
In Opinion

Scott Aaronson writes, “Assuming Google has achieved quantum supremacy, what exactly has it proved — and is it something anyone doubted in the first place?” https://t.co/lENiUbPjUB
--------------------------------------------------------------------------------
“There is barely any escape here, only a visit to a neighboring reality. This is a show about 

## Task 3: Search for Tweets

In [29]:
search_words = "wildfire -filter:retweets"
date_since = "2019-10-01"
cursor = tweepy.Cursor(api.search,
                       q=search_words,
                       lang="en",
                       since=date_since,
                       tweet_mode = "extended")

In [30]:
tweets = cursor.items(10)
for tweet in tweets:
    print('-' * 80)
    print(tweet.full_text)
    print(tweet.author.name)
    print(tweet.author.location)

--------------------------------------------------------------------------------
hollywoodphotog:⚠️The latest on #wildfire updates, #evacuation centers, power shutoffs
and road closures. 
#statewide #california

https://t.co/Jfl7zODioz Twitter https://t.co/Lnr3NvgqQ0)
EMS Daily
Worldwide
--------------------------------------------------------------------------------
“After Hurricane Katrina and Superstorm Sandy, both Congress and the FCC took a hard look at widespread outages," says @FreePressLeo. "[T]he FCC punted and put its faith in a voluntary regime devised by the industry.” https://t.co/hxI5Ao9REb
Dana Floberg
Washington, DC
--------------------------------------------------------------------------------
Goats help save Reagan library from destruction as California wildfire threatens

Flames from the Easy Fire were within a few feet of the Reagan library. Goats, which graze on grass around the property, helped prevent their spread.

https://t.co/4VLT7JNfWK https://t.co/yFdnA9sPR

In [31]:
# Create a Pandas DataFrame to store tweets, authors, and locations
import pandas as pd

# Create an empty data frame
tweets_df = pd.DataFrame(columns=['Name', 'Location', 'Text'])
tweets_df

,Name,Location,Text


In [32]:
# Append tweets data to the data frame
tweets = cursor.items(10)
idx = 0
for tweet in tweets:
    tweets_df.loc[idx, :] = [tweet.author.name,
                               tweet.author.location,
                               tweet.full_text]
    idx += 1
                               
tweets_df

,Name,Location,Text
0,The Progressive Mind,ProgressiveLand,Goats help save Reagan library from destructio...
1,Joy Hui Lin,"Los Angeles, CA","I literally asked a dating app guy, ""How's wil..."
2,Anyone but Trump 2020 🏳️‍🌈,"Tuscaloosa, AL",Fast-moving wildfire north of L.A. forces evac...
3,CJ,,Fast-moving wildfire north of L.A. forces evac...
4,Mistress Alexandra Wildfire 🔥,"London, England",Watch my ass while I give my slave a good poun...
5,Chris Shaffer,New York,Silicon Valley Leaders Sit Down With Wildfire ...
6,tensoba10,Hinter den sieben meeren Japan,New Wind-Driven Wildfire Erupts in Southern Ca...
7,Paul Stromberg,Chicago,@AudacityofGoats California's secret wildfire ...
8,Allan Johnson,"Minneapolis, MN",Fast-moving fire encroaches on Reagan Library ...
9,Tangee Lingard,,@realDonaldTrump @GreggJarrett There's a serio...


# II. Binary File Formats

## 1. pickle
The `pickle` module implements binary protocols for serializing and de-serializing a Python object structure. Only Python can properly read and write pickle files

In [33]:
# Let's create a data frame first
import numpy as np
import pandas as pd

values = np.array([
    [100, 80, 95, 'A'],
    [55, 60, 45, 'F'],
    [70, 75, 90, 'A'],
    [75, 70, 60, 'D'],
    [60, 73, 75, 'C'],
    [72, 63, -1, 'NA']
])
df = pd.DataFrame(values,
                   columns=['Midterm', 'Project', 'Final', 'LetterGrade'],
                   index=['Alex', 'Bob', 'Chris', 'Doug', 'Eva', "Frank"])
df

,Midterm,Project,Final,LetterGrade
Alex,100,80,95,A
Bob,55,60,45,F
Chris,70,75,90,A
Doug,75,70,60,D
Eva,60,73,75,C
Frank,72,63,-1,NA


In [34]:
# Save as a .pickle file
df.to_pickle("Data/temp/data.pickle")

In [35]:
# Load the pickle file
df_pickle = pd.read_pickle("Data/temp/data.pickle")
df_pickle

,Midterm,Project,Final,LetterGrade
Alex,100,80,95,A
Bob,55,60,45,F
Chris,70,75,90,A
Doug,75,70,60,D
Eva,60,73,75,C
Frank,72,63,-1,NA


## 2. HDF5
The "HDF" stands for "hierarchical data format". HDF5 can be a good choice for working with very large datasets that don't fit into memory, as you can efficiently read and write small sections of large arrays.

In [36]:
df = pd.DataFrame({
    'Col1': np.random.randn(100),
    'Col2': np.random.randn(100)
})
df.head(5)

,Col1,Col2
0,0.089893,-1.055004
1,0.316448,0.493639
2,0.412466,-0.242224
3,1.474262,-0.787625
4,-1.604593,0.732834


In [40]:
# The PyTable package may require update
!pip3 install --upgrade tables

Requirement already up-to-date: tables in c:\users\ch002\anaconda3\envs\tensorflow\lib\site-packages (3.6.1)


In [ ]:
df.to_hdf('Data/temp/data.h5', 'obj1', format='table')

In [ ]:
df_hdf5 = pd.read_hdf('Data/temp/data.h5', 'obj1', where=['index < 3'])
df_hdf5

## 3. feather
The feather format is adapted from the R statistical language. It has extremely high read and write performance.

In [44]:
!pip install -U pyarrow

In [57]:
import time
start = time.time()
df.to_feather('Data/temp/data.feather')
# df.to_pickle('Data/temp/data2.pickle')
end = time.time()
print("Time cost:", (end - start))

Time cost: 0.009720563888549805


In [61]:
import time
start = time.time()
df_feather = pd.read_feather('Data/temp/data.feather')
# df2_pickle = pd.read_pickle('Data/temp/data2.pickle')
end = time.time()
print("Time cost:", (end - start))
# df_feather

Time cost: 0.007938623428344727


# III. Interacting with Databases
In a business setting, most data may not be stored in text or binary files. SQL-based relational databases (such as mySQL) are in wide use.

Python has sqlite3 package to interact with databases, and Pandas has some functions to simplify the process.

In [63]:
# Create a SQLite database
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('Data/temp/data2.sqlite')
con.execute(query)
con.commit()

In [ ]:
# query = """
# DROP TABLE test
# """
# con.execute(query)
# con.commit()

In [64]:
# Insert a few rows of data
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [65]:
# Select data
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [66]:
# Retrieve columns names
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [68]:
# Extract column names from description

# Solution 1: use a loop
desc = cursor.description
cols = []
for elt in desc:
#     print(elt[0])
    cols.append(elt[0])
print(cols)

['a', 'b', 'c', 'd']


In [69]:
cols = [elt[0] for elt in cursor.description]
print(cols)

['a', 'b', 'c', 'd']


In [70]:
# Create a pandas data frame
columns = [x[0] for x in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


Use package `sqlalchemy` to create a data frame directly from a database.

In [72]:
!pip install sqlalchemy

  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.10-cp36-cp36m-win_amd64.whl size=1154938 sha256=fb69d6b1468e39cd2cc496855cd54f7ae3a8a07a8794d086093c5aee41a83ab5
  Stored in directory: C:\Users\ch002\AppData\Local\pip\Cache\wheels\4b\b2\89\cd2231ee623987c605f049df55f40a3e4252ef6a15b94836c2
Successfully built sqlalchemy


In [73]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///Data/temp/data.sqlite')
df = pd.read_sql('select * from test', db)
df

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
